## Background
A comparative analysis is needed to evaluate the performance and capabilities of ChatGPT 4.0 and Claude 3.5 Sonnet in converting Python code to optimized C++ code.

### Objectives
 - Convert existing Python codebase to C++
 - Optimize the converted code for better performance
 - Compare the effectiveness of AI assistants in code conversion and optimization
### Key Requirements
- Code Conversion Requirements
- Maintain functional equivalence with the original Python code
 - Properly handle memory management in C++
- Ensure type safety and appropriate C++ data structures
- Preserve code readability and maintainability
- Optimization Requirements
- Improve execution speed
- Reduce memory usage
- Implement C++-specific optimizations
- Utilize modern C++ features (C++11 and above)

In [2]:
from dotenv import load_dotenv
import os

In [3]:
# Load environment variables from .env file
load_dotenv()

# Get the environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [4]:
# system promt and user prompts
system_prompt = "You are an expert C++ software engineer specializing in system programming, optimization, and cross-language code conversion. Your task is to help convert and optimize a Python environment file loading system to C++, following C++ standards and best practices."
system_prompt += 'do not put commas or any character for the numbers'

In [5]:
def user_prompt_for(python):
    user_prompt = "Convert the following Python code to C++ code, following best practices, make sure the outputs of python code and c++ code should be accurate."
    user_prompt += "\n\nHere is the Python code:\n\n"
    user_prompt += python
    user_prompt += "\n\nNow, convert the Python code to C++ code, ensuring that the C++ code maintains the same functionality and produces the same output as the Python code. Make sure to handle memory management properly and use appropriate C++ data structures. Optimize the C++ code for better performance, considering factors such as execution speed and memory usage. check the syntax of the code before response"
    user_prompt += "\n\nPlease provide the optimized C++ code as the output. Do not give any text or explimation before and after c++ code. Give in such a way that the output is saved as a file and I should be able to run it without modifying"
    return user_prompt

In [6]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [7]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt_for(pi)}
]

In [35]:
import openai
from openai import OpenAI
import os
import sys

def stream_openai_response(model: str = "gpt-4"):
    """
    Stream responses from OpenAI API
    
    Args:
        messages: List of message dictionaries
        model: OpenAI model to use
    """
    try:
        # Initialize OpenAI client
        client = OpenAI(api_key=OPENAI_API_KEY)
        
        # Create streaming response
        stream = client.chat.completions.create(
            model=model,
            messages=messages,
            stream=True,
            temperature=0.7,
        )
        
        # Process the stream
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                # Clean the content by removing markdown code markers
                cleaned_content = content.replace('```cpp', '').replace('```', '')
                cleaned_content = cleaned_content.replace('cpp', '')
                print(cleaned_content, end='', flush=True)
                full_response += cleaned_content
                
        return full_response
                
    except openai.APIError as e:
        print(f"OpenAI API Error: {e}", file=sys.stderr)
        raise
    except Exception as e:
        print(f"Unexpected error: {e}", file=sys.stderr)
        raise

In [ ]:
stream_openai_response()

In [8]:
import anthropic
claude_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

def stream_claude_response(model: str = "claude-3-5-sonnet-20240620"):
    try:
        result = claude_client.messages.stream(
            model=model,
            max_tokens=2000,
            system=system_prompt,
            messages=[{"role": "user", "content": user_prompt_for(pi)}],
        )

        reply = ""
        with result as stream:
            for text in stream.text_stream:
                reply += text
                print(text, end="", flush=True)  # This will show the streaming response
        return reply

    except anthropic.APIError as e:
        print(f"Claude API Error: {e}", file=sys.stderr)
        raise
    except Exception as e:
        print(f"Unexpected error: {e}", file=sys.stderr)
        raise

In [ ]:
stream_claude_response()

In [9]:
def write_output(cpp, filename):
    code = cpp.replace("```cpp","").replace("```","")
    with open(f"{filename}.cpp", "w") as f:
        f.write(code)

In [ ]:
write_output(stream_claude_response(), "optimisedcode.cpp")

In [30]:
def modelSelection(model_choice):
    if model_choice == "Claude":
        return stream_claude_response()
    if model_choice == "GPT":
        return stream_openai_response()

In [33]:
import gradio as gr

def create_gradio_interface():
    with gr.Blocks() as app:
        with gr.Row():
            python_input = gr.Code(
                label="Python Code",
                language="python",
                value=pi,
                elem_classes=["code-box"]
            )
            cpp_output = gr.Code(
                label="C++ Code",
                language="cpp",
                interactive=True,
                elem_classes=["code-box"]
            )
        
        with gr.Row():
            model_choice = gr.Radio(
                choices=["Claude", "GPT"],
                value="Claude",
                label="Choose AI Model"
            )
        
        with gr.Row():
            translate_button = gr.Button("Translate")
        
        translate_button.click(
            fn=modelSelection,
            inputs=[model_choice],
            outputs=cpp_output
        )
    
    return app

In [ ]:
app = create_gradio_interface()
app.launch()

#### When analyzing the execution speed differences between Python and the AI-generated C++ code, we observed significant performance improvements. The C++ code produced by Claude demonstrated exceptional efficiency, running 60,000 times faster than its Python counterpart. Similarly, the C++ code generated by GPT also showed remarkable performance, executing 40,000 times faster than the original Python implementation. This stark contrast in runtime performance highlights the substantial speed advantages of the AI-translated C++ code over the original Python version.

- Version 1(Current): The existing Gradio interface features a dual-panel display system: one side shows the original Python code, while the other presents its AI-translated version. Users can toggle between Claude or GPT as their preferred translation model.

- Version 2(Next Steps): Future development will focus on embedding C++ compilation capabilities directly within the Gradio interface, allowing users to instantly compare performance metrics between the Python and C++ implementations.